In [ ]:
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import tensorflow as tf
import numpy as np
from pydub import AudioSegment

MODEL_PATH = "final_model.keras"
model = tf.keras.models.load_model(MODEL_PATH)

def friend_command(update: Update, context: CallbackContext) -> None:
 
    update.message.reply_text("Hello! Please send a voice message for prediction.")

def handle_voice(update: Update, context: CallbackContext) -> None:
   
    voice = update.message.voice or update.message.audio
    if not voice:
        update.message.reply_text("Please send a valid voice message.")
        return

 
    audio_file = voice.get_file()
    audio_path = "voice.ogg"
    audio_file.download(audio_path)

    
    audio = AudioSegment.from_file(audio_path)
    audio = audio.set_channels(1).set_frame_rate(48000)  
    processed_audio_path = "processed_audio.wav"
    audio.export(processed_audio_path, format="wav")

   
    input_data = np.array(audio.get_array_of_samples()).reshape(1, 48000, 1)

 
    try:
        prediction = model.predict(input_data)
        predicted_class = np.argmax(prediction, axis=1)[0] 
        update.message.reply_text(f"Predicted class: {predicted_class}")
    except Exception as e:
        update.message.reply_text(f"Error during prediction: {str(e)}")

def main():

    updater = Updater("TOKEN", use_context=True)
    dispatcher = updater.dispatcher

    #  /friend
    dispatcher.add_handler(CommandHandler("friend", friend_command))

   
    dispatcher.add_handler(MessageHandler(Filters.voice | Filters.audio, handle_voice))

    
    updater.start_polling()
    updater.idle()

if __name__ == "__main__":
    main()
